# setup

In [1]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2Model,AutoModel, AutoTokenizer
import os
from tqdm.auto import tqdm
import numpy as np
import re

from time import time
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.preprocessing import StandardScaler, LabelEncoder

import random

random.seed(42)


struct = 'lgbm'
struct = 'tabnet'

In [2]:
# データ読み込み
train_df = pd.read_csv("./train/train.csv")
test_df = pd.read_csv("./train/test.csv")
anime_df = pd.read_csv("./train/anime.csv")




inter = list(set(train_df.user_id.unique())&set(test_df.user_id.unique()))
print('recommend to:',len(inter))
test_outer = list(set(test_df.user_id.unique())-set(train_df.user_id.unique()))
print('need to predict:',len(inter))
# train_df = anime_info.merge(train_df,on='anime_id')
# test_df = anime_info.merge(test_df,on='anime_id')

do_embedding = True

train_df.head(5)

recommend to: 1780
need to predict: 1780


,user_id,anime_id,score
0,0008e10fb39e55447333,0669cc0219d468761195,2
1,0008e10fb39e55447333,111adb8835b8a1a2cf54,10
2,0008e10fb39e55447333,1fc8683c393432a2f9c7,1
3,0008e10fb39e55447333,2290175205d55e81b197,8
4,0008e10fb39e55447333,28f173b60331d5cabb0d,9


In [3]:
train_inter = train_df[train_df.user_id.isin(inter)]

test_inter = test_df[test_df.user_id.isin(inter)]
test_outer = test_df[test_df.user_id.isin(test_outer)]
# test_outer = test_outer[['user_id','anime_id']].merge(train_scores,on=['anime_id'])
# test_outer.to_csv('submission_outer.csv',index=False)
# test_inter.to_csv('test/test_inter.csv',index=False)
# test_outer.to_csv('test/test_outer.csv',index=False)
# train_inter.to_csv('train/train_inter.csv',index=False)
# train_outer.to_csv('train/train_outer.csv')
# train_outer.info()

# Preprocessing

In [4]:
def weighted_mean(x):
    counts = x.value_counts()
    total = counts.sum()
    weighted_sum = sum(score * count for score, count in counts.items())
    return weighted_sum / total

def to_minutes(s):

    match = re.search(r'(\d+) hr.*? (\d+) min.', s)
    if match:
        return int(match.group(1)) * 60 + int(match.group(2))
    match = re.search(r'(\d+) min', s)
    if match:
        return int(match.group(1))
    return 1 # for Unknown

def merge_by_anime_id(left_df, right_df):
    return pd.merge(left_df, right_df, on="anime_id", how="left").drop(columns=["anime_id","user_id"])

def merge_by_anime_id_with(left_df, right_df):
    return pd.merge(left_df, right_df, on="anime_id", how="left")


def merge_by_user_id_with(left_df, right_df):
    return pd.merge(left_df, right_df, on="user_id", how="left")

# train_scores = train_df.groupby('anime_id')['score'].apply(weighted_mean)
def create_anime_numeric_feature(input_df: pd.DataFrame):
    """input_dfは train or test.csv のデータが入ってくることを想定しています."""
    
    use_columns = [
        "members", 
        "watching","completed","on_hold","dropped","plan_to_watch"
    ]
    return merge_by_anime_id(input_df, anime_df)[use_columns]
    # return pd.merge(input_df, anime_df, on="anime_id", how="left").drop(columns=["anime_id","user_id"])


def create_anime_type_one_hot_encoding(input_df):
    
    target_colname = "type"
    target_series = anime_df[target_colname]
    unique_values = target_series.unique()

    out_df = pd.DataFrame()
    for value in unique_values:
        is_value = target_series == value
        out_df[value] = is_value.astype(int)
        
    out_df["anime_id"] = anime_df["anime_id"]
    
    return merge_by_anime_id(input_df, out_df)
def type(input_df):
    # 単純にラベルエンコーディング
    encoder = LabelEncoder()
    type_encoder = anime_df[['anime_id']].copy()
    type_encoder["type_label"] = encoder.fit_transform(anime_df["type"])
    return merge_by_anime_id(input_df, type_encoder)

def create_anime_type_one_hot_encoding(input_df):
    
    target_colname = "type"
    target_series = anime_df[target_colname]
    unique_values = target_series.unique()

    out_df = pd.DataFrame()
    for value in unique_values:
        is_value = target_series == value
        out_df[value] = is_value.astype(int)
        
    out_df["anime_id"] = anime_df["anime_id"]
    
    return merge_by_anime_id(input_df, out_df)

def create_anime_type_count_encoding(input_df):
    count = anime_df["type"].map(anime_df["type"].value_counts())
    encoded_df = pd.DataFrame({
        "anime_id": anime_df["anime_id"],
        "tyoe_count": count
    })
    
    return merge_by_anime_id(input_df, encoded_df)

def create_licensors_count_encoding(input_df):
    count = anime_df["licensors"].map(anime_df["licensors"].value_counts())
    encoded_df = pd.DataFrame({
        "anime_id": anime_df["anime_id"],
        "licensors_count": count
    })
    
    return merge_by_anime_id(input_df, encoded_df)

def studios(input_df):
    # 単純にラベルエンコーディング
    encoder = LabelEncoder()
    studios_encode = anime_df[['anime_id']].copy()
    studios_encode["studios"] = encoder.fit_transform(anime_df["studios"])
    return merge_by_anime_id(input_df, studios_encode)

def create_studio_count_encoding(input_df):
    count = anime_df["studios"].map(anime_df["studios"].value_counts())
    encoded_df = pd.DataFrame({
        "anime_id": anime_df["anime_id"],
        "studios_count": count
    })
    
    return merge_by_anime_id(input_df, encoded_df)

def create_source_count_encoding(input_df):
    count = anime_df["source"].map(anime_df["source"].value_counts())
    encoded_df = pd.DataFrame({
        "anime_id": anime_df["anime_id"],
        "source_count": count
    })
    
    return merge_by_anime_id(input_df, encoded_df)

def source(input_df):
    # 単純にラベルエンコーディング
    encoder = LabelEncoder()
    source_encode = anime_df[['anime_id']].copy()
    source_encode["source"] = encoder.fit_transform(anime_df["source"])
    return merge_by_anime_id(input_df, source_encode)

def create_rating_count_encoding(input_df):
    count = anime_df["rating"].map(anime_df["rating"].value_counts())
    encoded_df = pd.DataFrame({
        "anime_id": anime_df["anime_id"],
        "rating_count": count
    })
    
    return merge_by_anime_id(input_df, encoded_df)

def create_genres_onehot_encoding(input_df):
    """Create 26-dim embedding"""
    chars = ['Comedy', 'Sci-Fi', 'Seinen', 'Slice of Life', 'Space',
       'Adventure', 'Mystery', 'Historical', 'Supernatural', 'Fantasy',
       'Ecchi', 'School', 'Harem', 'Romance', 'Shounen', 'Action',
       'Magic', 'Sports', 'Super Power', 'Drama', 'Thriller', 'Music',
       'Shoujo', 'Demons', 'Mecha', 'Game', 'Josei', 'Cars',
       'Psychological', 'Parody', 'Samurai', 'Military', 'Shoujo Ai',
       'Kids', 'Martial Arts', 'Horror', 'Dementia', 'Vampire',
       'Shounen Ai', 'Hentai', 'Yaoi', 'Police']
    genres = anime_df[['anime_id','genres']]
    genres.loc[:,chars] = 0
    genres['genres'] = genres['genres'].str.split(',')
    # genres[chars] = 0
    for i, row in tqdm(genres.iterrows(),total=2000):
        for index in (s.strip() for s in row['genres']):
                genres.loc[i,index] = 1
    genres = genres.drop('genres',axis=1)
    return merge_by_anime_id(input_df, genres)


def create_producer_onehot_encoding(input_df):
    """Create 26-dim embedding"""
    producer = anime_df['producers'].str.split(',')
    all_producer = list(map(str.strip,producer.explode().unique()))
    producer = anime_df[['anime_id']]
    producer.loc[:,all_producer] = 0
    producer['producers'] = anime_df['producers'].str.split(',')
    # genres[chars] = 0
    for i, row in tqdm(producer.iterrows(),total=2000):
        for index in (s.strip() for s in row['producers']):
                producer.loc[i,index] = 1
    
    return merge_by_anime_id(input_df, producer).drop('producers',axis=1)


def create_anime_type_one_hot_encoding(input_df):
    
    target_colname = "type"
    target_series = anime_df[target_colname]
    unique_values = target_series.unique()

    out_df = pd.DataFrame()
    for value in unique_values:
        is_value = target_series == value
        out_df[value] = is_value.astype(int)
        
    out_df["anime_id"] = anime_df["anime_id"]
    
    return merge_by_anime_id(input_df, out_df)

def create_duration2min(input_df):
    time_min = anime_df[['anime_id']]
    time_min['minutes'] = anime_df['duration'].apply(to_minutes)
    return merge_by_anime_id(input_df, time_min)
from datetime import datetime
def to_year(s):
    match = re.search(r'\d{4}', s)
    if match:
        return int(match.group())
    else:
        return 1000
    
def year_pre(input_df):
    year = anime_df[['anime_id']]
    encoder = LabelEncoder()
    year['year'] = encoder.fit_transform(anime_df['aired'].apply(to_year))
    
    return merge_by_anime_id(input_df, year)

def merge_embedding(input_df):
    embeds = np.load("train/anime_jpbert_embeddings.npy")
    ids = np.load("train/anime_jpbert_ids.npy")
    embeds_list = []
    for l in range(embeds.shape[0]):
        embeds_list.append(embeds[l,:])
    anime_embed = pd.DataFrame(data={"anime_id": ids, "embed" : embeds_list})
    embed_array = np.array(anime_embed["embed"].tolist())

    for i in range(768):
        anime_embed[f"embed_{i}"] = embed_array[:,i]

    anime_embed.drop("embed", axis=1, inplace=True)
    return merge_by_anime_id(input_df, anime_embed)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
# from allennlp.modules.elmo import Elmo, batch_to_ids
from scipy.spatial.distance import pdist, squareform
import pickle
from gensim.models import word2vec,Word2Vec

def create_tfidf_matrix(input_df):
    text_columns = ['genres', 'japanese_name', 'type', 'episodes', 'aired', 'producers', 'licensors', 'studios', 'source', 'duration', 'rating']

    tf = TfidfVectorizer()
    try:
        tf = pickle.load(open("vectorizer.pickle", "rb"))
        print('loaded tf')
    except:
        tf.fit(anime_df[text_columns].astype(str).apply(lambda x: ';'.join(x), axis=1).tolist()) 
        pickle.dump(tf, open("vectorizer.pickle", "wb"))
    tfidf_matrix = tf.fit_transform(anime_df[text_columns].astype(str).apply(lambda x: ';'.join(x), axis=1).tolist())
    cosine_sim = cosine_similarity(tfidf_matrix) 
    cosine_sim_df = pd.DataFrame(cosine_sim, index=anime_df['anime_id'],columns=anime_df['anime_id'])
    svd = TruncatedSVD(n_components=10,random_state=42)

    svd_arr = svd.fit_transform(cosine_sim_df)
    col_df = pd.DataFrame(
    svd_arr,
    index=cosine_sim_df.index,
    columns=[f"svd_{ix}" for ix in range(10)],
    ).reindex()
    return merge_by_anime_id_with(input_df, col_df)

def create_user_anime_vector(input_df):
    df = pd.concat([train_df,test_df])
    if 'score' in input_df.columns:
        user_matrix = create_tfidf_matrix(df).drop(['anime_id','score'],axis=1).groupby('user_id').mean().reindex()
    else:
        user_matrix = create_tfidf_matrix(df).drop(['anime_id'],axis=1).groupby('user_id').mean().reindex()
    
    return merge_by_user_id_with(input_df, user_matrix).drop('user_id',axis=1)

def create_01EDA_user_vector(input_df):
    df = pd.concat([train_df,test_df])
    if 'score' in input_df.columns:
        user_matrix = create_llma_anime_vector(df,embed_source='mBERT_embedding_01EDA',mode='user').drop(['anime_id','score'],axis=1).groupby('user_id').mean().reindex()
    else:
        user_matrix = create_llma_anime_vector(df,embed_source='mBERT_embedding_01EDA',mode='user').drop(['anime_id'],axis=1).groupby('user_id').mean().reindex()
    
    return merge_by_user_id_with(input_df, user_matrix).drop('user_id',axis=1)

def create_large_user_vector(input_df):
    df = pd.concat([train_df,test_df])
    if 'score' in input_df.columns:
        user_matrix = create_llma_anime_vector(df,embed_source='mBERT_embedding_large',mode='user').drop(['anime_id','score'],axis=1).groupby('user_id').mean().reindex()
    else:
        user_matrix = create_llma_anime_vector(df,embed_source='mBERT_embedding_large',mode='user').drop(['anime_id'],axis=1).groupby('user_id').mean().reindex()
    
    return merge_by_user_id_with(input_df, user_matrix).drop('user_id',axis=1)

def create_small_user_vector(input_df):
    df = pd.concat([train_df,test_df])
    if 'score' in input_df.columns:
        user_matrix = create_llma_anime_vector(df,embed_source='mBERT_embedding_small',mode='user').drop(['anime_id','score'],axis=1).groupby('user_id').mean().reindex()
    else:
        user_matrix = create_llma_anime_vector(df,embed_source='mBERT_embedding_small',mode='user').drop(['anime_id'],axis=1).groupby('user_id').mean().reindex()
    
    return merge_by_user_id_with(input_df, user_matrix).drop('user_id',axis=1)

def create_llma_anime_vector(input_df,embed_source,mode='anime'):
    embeddings = np.load(f"./train/{embed_source}.npy")
    df_anime_preprocessd = anime_df[['anime_id']].copy()
    # cosine_sim = cosine_similarity(embeddings)
    # svd = TruncatedSVD(n_components=20,random_state=42)
    # svd_arr = svd.fit_transform(cosine_sim)
    
    # df_anime_preprocessd = df_anime_preprocessd.drop(columns=["combined_features"])
    embeddings_columns = [f"{embed_source}_{mode}_{i}" for i in range(embeddings.shape[1])]
    embeddings_df = pd.DataFrame(data=embeddings, columns=embeddings_columns)
    df_anime_preprocessd = df_anime_preprocessd.join(embeddings_df)
    
    return merge_by_anime_id_with(input_df, df_anime_preprocessd)

def create_01EDA_anime_vector(input_df):
    return create_llma_anime_vector(input_df,embed_source='mBERT_embedding_01EDA').drop(columns=["anime_id","user_id"])
def create_large_anime_vector(input_df):
    return create_llma_anime_vector(input_df,embed_source='mBERT_embedding_large').drop(columns=["anime_id","user_id"])
def create_small_anime_vector(input_df):
    return create_llma_anime_vector(input_df,embed_source='mBERT_embedding_small').drop(columns=["anime_id","user_id"])

def create_w2v_anime_vector(input_df,mode='anime'):
    text_columns = ['genres', 'japanese_name', 'type', 'episodes', 'aired', 'producers', 'licensors', 'studios', 'source']
    text_data = anime_df[text_columns].astype(str).apply(lambda x: ';;'.join(x), axis=1).tolist()

    text1 = [x.strip() for _ in text_data for x in re.split(';;',_) ] # テキストデータをコンマやセミコロンで分割して、空白を除去する
    text_data.append(text1)
    shuffled_sentence_list = [random.sample(sentence, len(sentence)) for sentence in text_data] 
    train_sentence_list = text_data + shuffled_sentence_list
    vector_size = 128
    w2v_params = {
        "vector_size": vector_size,  ## <= 変更点
        "seed": 42,
        "min_count": 1,
        "workers": 1
    }

    # word2vecのモデル学習
    try:
        model = Word2Vec.load('model/w2v.model')
    except:
        model = word2vec.Word2Vec(train_sentence_list, **w2v_params)
        model.save('model/w2v.model')
    vects = {}
    for i, row in anime_df.iterrows():
        vect = []
        for col in text_columns:
            vect.append(model.wv[row[col]])
        vects[row.anime_id] = np.concatenate(vect,axis=0)
    wv_matirx = pd.DataFrame(vects).T
    # svd = TruncatedSVD(n_components=20,random_state=42)

    # svd_cos = svd.fit_transform(cosine_similarity(wv_matirx))
    wv_cos = pd.DataFrame(wv_matirx,
                          index=wv_matirx.index.values.tolist(),
                          columns=[f"wv_{mode}_cos_vec_{i}" for i in range(wv_matirx.shape[1])]
                          )
    wv_cos.index.name = 'anime_id'
    return merge_by_anime_id_with(input_df,wv_cos.reindex())

def create_w2v_user_vector(input_df):
    df = pd.concat([train_df,test_df])
    if 'score' in input_df.columns:
        user_matrix = create_w2v_anime_vector(df,mode='user').drop(['anime_id','score'],axis=1).groupby('user_id').mean().reindex()
    else:
        user_matrix = create_w2v_anime_vector(df,mode='user').drop(['anime_id'],axis=1).groupby('user_id').mean().reindex()
    
    return merge_by_user_id_with(input_df, user_matrix).drop('user_id',axis=1)

In [5]:
def create_feature(input_df):
    
    # functions に特徴量作成関数を配列で定義しました.
    # どの関数も同じ input / output のインターフェイスなので for で回せて嬉しいですね ;)
    functions = [
        create_user_anime_vector,
        create_tfidf_matrix,
        create_w2v_anime_vector,

        create_w2v_user_vector,
        create_01EDA_user_vector,
        create_large_user_vector,
        create_small_user_vector,

        create_anime_numeric_feature,
        
        type,
        studios,
        source,
        # create_anime_type_count_encoding,
        
        # create_anime_type_one_hot_encoding,
        create_genres_onehot_encoding,
        # create_producer_onehot_encoding,
        # create_licensors_count_encoding,
    
        # create_studio_count_encoding,
        # create_source_count_encoding,
        create_rating_count_encoding,
        create_duration2min,

        create_01EDA_anime_vector,
        create_large_anime_vector,
        create_small_anime_vector,
        
        year_pre,
    ]
    
    out_df = pd.DataFrame()
    for func in functions:
        func_name = str(func.__name__)

        _df = func(input_df)
        out_df = pd.concat([out_df, _df], axis=1)
        
    return out_df

In [6]:
# test_df['score'] = 0
# train_test_df = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)
# train_test_df = add_w2v_features_without_score(train_test_df)
# train_df = train_test_df[train_test_df['score'] >= 0].copy().reset_index(drop=True)
# test_df = train_test_df[train_test_df['score'] == 0].copy().reset_index(drop=True)

In [7]:
# train_df, test_df = add_w2v_features(train_df, test_df)
if struct == 'lgbm':
    train_feat_df = create_feature(train_df)
    test_feat_df = create_feature(test_df)
    for text_col in ['anime_id','score','user_id']:
        if text_col in train_feat_df.columns:
            train_feat_df = train_feat_df.drop(columns=text_col)
        if text_col in test_feat_df.columns:
            test_feat_df = test_feat_df.drop(columns=text_col)
    X = train_feat_df.values
    y = train_df["score"].values

In [8]:
# train_feat_df.head(5)

In [9]:
import torch
import numpy as np
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error


class TabNetBaseline:
    def __init__(
        self,
        tabnet_params,
        embedding_cols,
        embedding_idx,
        cat_dims,
        embedding_dims,
        splitter=KFold(n_splits=5, shuffle=True, random_state=42),
        seed=777
        ):
        self.tabnet_params = tabnet_params
        self.embedding_cols = embedding_cols
        self.embedding_idx = embedding_idx
        self.cat_dims = cat_dims
        self.embedding_dims = embedding_dims
        self.splitter = splitter
        self.seed = seed
        self.models = []
        self.oof_preds = None
        self.pretrained_model = None

    def prepare_pretrain_data(self, df_pretrain):

        for embc, d in zip(self.embedding_cols, self.cat_dims):
            df_pretrain[embc] = df_pretrain[embc].replace({-1: df_pretrain[embc].max() + 1})
            assert (df_pretrain[embc].max()+1 == d)
            assert (len(df_pretrain[embc].unique()) == d)
            assert df_pretrain[embc].min()==0

        return df_pretrain

    def pretrain(self, df_pretrain):
        if not self.pretrained_model:
            df_pretrain = self.prepare_pretrain_data(df_pretrain)
            train_unsp, val_unsup = train_test_split(df_pretrain,  test_size=0.3, random_state=self.seed)

            unsupervised_model = TabNetPretrainer(
                optimizer_fn=torch.optim.Adam,
                optimizer_params=dict(lr=0.1),
                scheduler_params={"step_size":10, "gamma":0.9},
                cat_idxs=self.embedding_idx, 
                cat_dims=self.cat_dims,
                cat_emb_dim=self.embedding_dims,
                **self.tabnet_params
            )

            unsupervised_model.fit(
                X_train=train_unsp.values,
                eval_set=[val_unsup.values],
                pretraining_ratio=0.8,
                max_epochs=300
            )

            self.pretrained_model = unsupervised_model
        return self.pretrained_model

    def train(self, X, y, groups=None):
        self.models = []
        self.oof_preds = np.zeros(len(X))
        scores = []
        for train_index, valid_index in self.splitter.split(X, y, groups=groups):
            unsupervised_model = self.pretrained_model

            X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

            reg = TabNetRegressor(
                optimizer_fn=torch.optim.Adam,
                optimizer_params=dict(lr=0.03),
                scheduler_params={"step_size":10, "gamma":0.9},
                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                cat_idxs=self.embedding_idx, 
                cat_dims=self.cat_dims,
                cat_emb_dim=self.embedding_dims,
                **self.tabnet_params
            )

            reg.fit(
                X_train=X_train.values, y_train=y_train.values.reshape(-1, 1),
                eval_set=[(X_train.values, y_train.values.reshape(-1, 1)), (X_valid.values, y_valid.values.reshape(-1, 1))],
                eval_name=['train', 'valid'],
                eval_metric=['rmse'],
                batch_size=2048, virtual_batch_size=2048,
                drop_last=True,
                from_unsupervised=unsupervised_model,
                max_epochs=1000,
                patience=12,
                num_workers=4,
            )
            self.models.append(reg)

            y_pred = reg.predict(X_valid.values)[:, 0]
            self.oof_preds[valid_index] = y_pred
            score = mean_squared_error(y_valid, y_pred, squared=False)  # RMSE score
            scores.append(score)
        self.cv_score = np.mean(scores)

    def inference(self, X):
        y_preds = []
        for model in self.models:
            y_pred = model.predict(X)[:, 0]
            y_preds.append(y_pred)
        y_preds = np.mean(y_preds, axis=0)
        return y_preds

    def plot_feature_importance(self):
        df_features_list = []
        for model in self.models:
            df = pd.DataFrame(data ={
                "feature_importance" : model.feature_importances_,
                "feature_names" : self.features
            })
            df_features_list.append(df)

        df_features = pd.concat(df_features_list).sort_values(by='feature_importance', ascending=False)

        f, ax = plt.subplots(figsize=(5, 10))
        sns.barplot(
            data = df_features,
            x = 'feature_importance',
            y = 'feature_names',
            capsize=0.1, errwidth=1.2,
            ax = ax
        )
        return f, ax
# Pretrainerのハイパーパラメータ
tabnet_params = {
    'mask_type' : 'entmax',
    'n_d' : 64,
    'n_a': 64,
    'n_steps': 3,
    'gamma': 0.9,
    'verbose': 1
}

In [10]:
def concat_train_test(train, test):
    # train["train_test"] = "train"
    # test["train_test"] = "test"
    df_all = pd.concat([train, test])
    return df_all

In [11]:
if struct == 'tabnet':
    train_feat_df = create_feature(train_df)
    test_feat_df = create_feature(test_df)
    
    # df_pretrain = create_feature(concat_train_test(train_df,test_df))
    for text_col in ['anime_id','score','user_id']:
        if text_col in train_feat_df.columns:
            train_feat_df = train_feat_df.drop(columns=text_col)
        if text_col in test_feat_df.columns:
            test_feat_df = test_feat_df.drop(columns=text_col)
    df_pretrain = concat_train_test(train_feat_df,test_feat_df)

loaded tf
loaded tf


C:\Users\Horikita_Saku\AppData\Local\Temp\ipykernel_26396\934449817.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genres.loc[:,chars] = 0
C:\Users\Horikita_Saku\AppData\Local\Temp\ipykernel_26396\934449817.py:145: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  genres.loc[:,chars] = 0
C:\Users\Horikita_Saku\AppData\Local\Temp\ipykernel_26396\934449817.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

  0%|          | 0/2000 [00:00<?, ?it/s]

C:\Users\Horikita_Saku\AppData\Local\Temp\ipykernel_26396\934449817.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_min['minutes'] = anime_df['duration'].apply(to_minutes)
C:\Users\Horikita_Saku\AppData\Local\Temp\ipykernel_26396\934449817.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year['year'] = encoder.fit_transform(anime_df['aired'].apply(to_year))


loaded tf


KeyboardInterrupt: 

In [13]:
text_columns = ['genres', 'japanese_name', 'type', 'episodes', 'aired', 'producers', 'licensors', 'studios', 'source', 'duration', 'rating']

tf = TfidfVectorizer()
try:
    tf = pickle.load(open("vectorizer.pickle", "rb"))
    print('loaded tf')
except:
    tf.fit(anime_df[text_columns].astype(str).apply(lambda x: ';'.join(x), axis=1).tolist()) 
    pickle.dump(tf, open("vectorizer.pickle", "wb"))
tfidf_matrix = tf.fit_transform(anime_df[text_columns].astype(str).apply(lambda x: ';'.join(x), axis=1).tolist())
# cosine_sim = cosine_similarity(tfidf_matrix) 
# cosine_sim_df = pd.DataFrame(tfidf_matrix, index=anime_df['anime_id'],columns=anime_df['anime_id'])
# svd = TruncatedSVD(n_components=10,random_state=42)

loaded tf


In [ ]:
df_pretrain[[
        'type_label',
        # 'studios',
        'source',
        # 'year'
    ]]

In [ ]:
# embeddingするカラム
embeding_cols = [
        'type_label',
        # 'studios',
        'source',
        # 'year'
    ]
# 各々のユニーク数
col_uniques = [ len(df_pretrain[c].unique()) for c in  embeding_cols ]
# カラムの番号
embeding_idx = [ i for i, c in  enumerate(df_pretrain.columns) if c in embeding_cols]
cat_dims = [ v for k, v in zip(embeding_cols, col_uniques) if k in embeding_cols]
# 埋め込み次元
embeding_dims = [ i // 2 if i<100 else 64 for i in cat_dims ]

In [ ]:
tabnet_mbert = TabNetBaseline(
    tabnet_params=tabnet_params,
    embedding_cols=embeding_cols,
    embedding_idx=embeding_idx,
    cat_dims=cat_dims,
    embedding_dims=embeding_dims
)


In [ ]:
# 保存
tabnet_mbert.pretrain(df_pretrain)
tabnet_mbert.pretrained_model.save_model('model/pretrain_mBERT')

In [ ]:
# train_x = df_merge_all.loc[df_merge_all["train_test"]=="train"].copy().drop(columns=drop_cols)
# train_y = df_merge_all.loc[df_merge_all["train_test"]=="train"].copy()['score']
# X = train_feat_df.values
# y = train_df["score"].values

loaded_pretrain = TabNetPretrainer()
loaded_pretrain.load_model('.pretrain_mBERT.zip')
tabnet_mbert.pretrained_model = loaded_pretrain
train_y = train_df.copy()['score'].astype(float)
tabnet_mbert.train(train_feat_df.astype(float),train_y)

# Train

In [ ]:
from sklearn.model_selection import StratifiedKFold

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv = fold.split(X, y)
cv = list(cv) # split の返り値は generator なので list 化して何度も iterate できるようにしておく

from sklearn.metrics import mean_squared_error

def root_mean_squared_error(y_true, y_pred):
    """mean_squared_error の root (0.5乗)"""
    return mean_squared_error(y_true, y_pred) ** .5
import lightgbm as lgbm
import xgboost
def fit_lgbm(X, 
             y, 
             cv, 
             params: dict=None, 
             verbose: int=50):
    """lightGBM を CrossValidation の枠組みで学習を行なう function"""

    # パラメータがないときは、空の dict で置き換える
    if params is None:
        params = {}

    models = []
    n_records = len(X)
    # training data の target と同じだけのゼロ配列を用意
    oof_pred = np.zeros((n_records, ), dtype=np.float32)

    for i, (idx_train, idx_valid) in enumerate(tqdm(cv)): 
        # この部分が交差検証のところです。データセットを cv instance によって分割します
        # training data を trian/valid に分割
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        clf = lgbm.LGBMRegressor(**params,early_stopping_rounds=500)


            
        # cv 内で train に定義された x_train で学習する
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)

        # cv 内で validation data とされた x_valid で予測をして oof_pred に保存していく
        # oof_pred は全部学習に使わなかったデータの予測結果になる → モデルの予測性能を見る指標として利用できる
        pred_i = clf.predict(x_valid)
        oof_pred[idx_valid] = pred_i
        models.append(clf)
        score = root_mean_squared_error(y_valid, pred_i)
        print(f" - fold{i + 1} - {score:.4f}")

    score = root_mean_squared_error(y, oof_pred)

    print("=" * 50)
    print(f"FINISHI: Whole Score: {score:.4f}")
    return oof_pred, models
params = {
    # 目的関数. これの意味で最小となるようなパラメータを探します. 
    "objective": "rmse", 

    # 木の最大数. early_stopping という枠組みで木の数は制御されるようにしていますのでとても大きい値を指定しておきます.
    "n_estimators": 20000, 
    
     # 学習率. 小さいほどなめらかな決定境界が作られて性能向上に繋がる場合が多いです、
    # がそれだけ木を作るため学習に時間がかかります
    "learning_rate": 0.05,
    "reg_alpha":1e-5,
    "reg_lambda":1e-5,

    # 特徴重要度計算のロジック(後述)
    "importance_type": "gain", 
    "random_state": 42,
}



In [ ]:
oof, models = fit_lgbm(X, y=y, params=params, cv=cv)

In [ ]:
root_mean_squared_error(y_true=y, y_pred=oof)

In [ ]:

def visualize_importance(models, feat_train_df):
    """lightGBM の model 配列の feature importance を plot する
    CVごとのブレを boxen plot として表現します.

    args:
        models:
            List of lightGBM models
        feat_train_df:
            学習時に使った DataFrame
    """
    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df["feature_importance"] = model.feature_importances_
        _df["column"] = feat_train_df.columns
        _df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], 
                                          axis=0, ignore_index=True)

    order = feature_importance_df.groupby("column")\
        .sum()[["feature_importance"]]\
        .sort_values("feature_importance", ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(12, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df, 
                  x="feature_importance", 
                  y="column", 
                  order=order, 
                  ax=ax, 
                  palette="viridis", 
                  orient="h")
    ax.tick_params(axis="x", rotation=90)
    ax.set_title("Importance")
    ax.grid()
    fig.tight_layout()
    return fig, ax
fig, ax = visualize_importance(models, train_feat_df)

In [ ]:
# k 個のモデルの予測を作成. shape = (5, N_test,).
pred = np.array([model.predict(test_feat_df.values) for model in tqdm(models)])
# k 個のモデルの予測値の平均 shape = (N_test,).
pred = np.mean(pred, axis=0) # axis=0 なので shape の `k` が潰れる 

In [ ]:
score = pd.DataFrame({
    "score": pred
})
score['score'] = score['score'].apply(lambda x: np.clip(x, 1, 10))
score.to_csv("submission.csv", index=False)

In [ ]:
test_df['score'] = pred
test_df['score'] = test_df['score'].apply(lambda x: np.clip(x, 1, 10))
test_df.to_csv("submission_outer.csv", index=False)

In [ ]:
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2Model.from_pretrained('./model')
model = AutoModel.from_pretrained("cl-tohoku/bert-base-japanese")
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
text_columns = ['genres', 'japanese_name', 'type', 'episodes', 'aired', 'producers', 'licensors', 'studios', 'source', 'duration', 'rating', 'members', 'watching', 'completed', 'on_hold', 'dropped', 'plan_to_watch']

# text_columns = ['japanese_name']
def get_gpt2_embeds(text):
    input_ids = tokenizer.encode(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(input_ids)
        embeddings = outputs[0][0][0]
    return embeddings.detach().cpu().numpy()
def concat_with_column_names(row):
    return ' '.join([f'{row[col]}' for col in text_columns])
    return ' '.join([f'{col}: {row[col]}' for col in text_columns])
if do_embedding == True:
    ids_list = []
    embeddings = []
    for i, row in tqdm(anime_df.iterrows(),total=2000):
        ids_list.append(row['anime_id'])
        text = concat_with_column_names(row[text_columns])
        embedding = get_gpt2_embeds(text)
        embeddings.append(embedding)
    np.save('train/anime_jpbert_ids.npy',np.array(ids_list))
    np.save('train/anime_jpbert_embeddings.npy',np.array(embeddings))
    print(len(embedding))

# Concat Sub

In [ ]:
import pandas as pd
inter_sub = pd.read_csv('submission_inner.csv')
outer_sub = pd.read_csv('submission_outer.csv')
all_sub = pd.concat([inter_sub,outer_sub])
sub = pd.read_csv('test/test.csv')
submission = sub.merge(all_sub,on=['user_id','anime_id'],how='left')
submission['score'] = submission['score'].apply(lambda x: np.clip(x, 1, 10))
submission['score'].to_csv('submission.csv',index=False)

In [ ]:
set(sub.anime_id.unique())-set(all_sub.anime_id.unique())

In [ ]:
pd.read_csv('submission.csv').score.to_csv('sub.csv',index=False)